In [ ]:
import tkinter as tk
from tkinter import messagebox
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
import base64

# Function to pad the text to be a multiple of 16 bytes
def pad(text):
    padding_length = 16 - (len(text) % 16)
    return text + (chr(padding_length) * padding_length)

# Function to remove padding
def unpad(text):
    return text[:-ord(text[-1])]

# Encryption function
def encrypt_text():
    text = entry_text.get("1.0", tk.END).strip()
    key = entry_key.get().encode()
    
    if len(key) != 16:
        messagebox.showerror("Error", "Key must be 16 bytes long!")
        return
    
    iv = os.urandom(16)  # Generate a random IV
    cipher = Cipher(algorithms.Camellia(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padded_text = pad(text).encode()
    encrypted = encryptor.update(padded_text) + encryptor.finalize()
    result = base64.b64encode(iv + encrypted).decode()
    
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, result)

# Decryption function
def decrypt_text():
    encrypted_text = entry_text.get("1.0", tk.END).strip()
    key = entry_key.get().encode()
    
    if len(key) != 16:
        messagebox.showerror("Error", "Key must be 16 bytes long!")
        return
    
    try:
        print("Ciphertext Before Decoding:", encrypted_text)
        decoded_data = base64.b64decode(encrypted_text + "===")  # Auto-fix padding
        print("Decoded Data Length:", len(decoded_data))
        
        if len(decoded_data) < 17:  # IV (16 bytes) + at least 1 byte ciphertext
            messagebox.showerror("Error", "Decryption failed! Encrypted data is too short.")
            return

        iv = decoded_data[:16]
        encrypted = decoded_data[16:]
        
        cipher = Cipher(algorithms.Camellia(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        
        decrypted_padded = decryptor.update(encrypted) + decryptor.finalize()
        decrypted = unpad(decrypted_padded.decode())

        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, decrypted)
    except Exception as e:
        print("Decryption Error:", str(e))
        messagebox.showerror("Error", "Decryption failed! Invalid key or data.")

# GUI setup
root = tk.Tk()
root.title("Camellia-128-CBC Cipher")
root.geometry("500x400")

label_key = tk.Label(root, text="Enter 16-byte Key:")
label_key.pack()
entry_key = tk.Entry(root, show="*")
entry_key.pack()

label_text = tk.Label(root, text="Enter Text to Encrypt/Decrypt:")
label_text.pack()
entry_text = tk.Text(root, height=5)
entry_text.pack()

encrypt_button = tk.Button(root, text="Encrypt", command=encrypt_text)
encrypt_button.pack()

decrypt_button = tk.Button(root, text="Decrypt", command=decrypt_text)
decrypt_button.pack()

label_output = tk.Label(root, text="Output:")
label_output.pack()
output_text = tk.Text(root, height=5)
output_text.pack()

root.mainloop()